In [0]:
from pyspark.sql.functions import *

In [0]:
df_covid_bronze = spark.read.format("delta").load("/mnt/bronze/covid19")
df_policyData_bronze = spark.read.format("delta").load("/mnt/bronze/policyData")
df_mobilityData_bronze = spark.read.format("delta").load("/mnt/bronze/mobilityData")

In [0]:
df_joined = df_covid_bronze\
    .join(df_policyData_bronze, on=['location', 'date'],how='left')\
    .join(df_mobilityData_bronze, on=['location','date'],how='left')

In [0]:
handle_nulls = {"total_cases": 0,"new_cases": 0,"total_deaths": 0,"new_deaths": 0,"total_tests": 0,
    "new_tests": 0,"total_vaccinations": 0,"people_vaccinated": 0,"people_fully_vaccinated": 0,"stringencyindex": 0,"reproduction_rate": -1,"c1_school_closing": -1,"c2_workplace_closing": -1,
    "c6_stay_at_home_requirements": -1,"h2_testing_policy": -1,"h3_contact_tracing": -1,
    "retail_and_recreation_percent_change_from_baseline": 0,
    "grocery_and_pharmacy_percent_change_from_baseline": 0,"workplaces_percent_change_from_baseline": 0,
    "residential_percent_change_from_baseline": 0,
    "c8_international_travel_controls": -1}

In [0]:
df_joined_2 = df_joined\
    .withColumn("year",year("date"))\
    .withColumn("month",month("date"))\
    .drop("stringency_index")\
    .fillna(handle_nulls)


In [0]:
rename_cols = {
    "total_cases": "covid_total_cases",
    "new_cases": "covid_new_cases",
    "total_deaths": "covid_total_deaths",
    "new_deaths": "covid_new_deaths",
    "reproduction_rate": "covid_reproduction_rate",
    "total_tests": "covid_total_tests",
    "new_tests": "covid_new_tests",
    "total_vaccinations": "covid_total_vax",
    "people_vaccinated": "covid_people_vax",
    "people_fully_vaccinated": "covid_fully_vax",
    "c1_school_closing": "policy_school_close",
    "c2_workplace_closing": "policy_work_close",
    "c6_stay_at_home_requirements": "policy_stay_home",
    "c8_international_travel_controls": "policy_travel_control",
    "h2_testing_policy": "policy_test_policy",
    "h3_contact_tracing": "policy_trace",
    "stringencyindex": "policy_stringency",
    "retail_and_recreation_percent_change_from_baseline": "mobility_retail",
    "grocery_and_pharmacy_percent_change_from_baseline": "mobility_grocery",
    "workplaces_percent_change_from_baseline": "mobility_workplace",
    "residential_percent_change_from_baseline": "mobility_home"
}

In [0]:
df_joined_3 = df_joined_2
for old_col,new_col in rename_cols.items():
    df_joined_3 = df_joined_3.withColumnRenamed(old_col,new_col)

In [0]:
df_joined_3.write \
    .format("delta") \
    .mode("overwrite") \
    .partitionBy("year", "month") \
    .save("/mnt/silver/covid_combined")